In [1]:
import findspark
findspark.init()
findspark.find()
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
range = spark.range(1000).toDF("number")
range.show()
range.show(5)  #show 5 rows

divBy3 = range.where("number % 3 == 0")
divBy3
#shows a new dataframe

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
|     5|
|     6|
|     7|
|     8|
|     9|
|    10|
|    11|
|    12|
|    13|
|    14|
|    15|
|    16|
|    17|
|    18|
|    19|
+------+
only showing top 20 rows

+------+
|number|
+------+
|     0|
|     1|
|     2|
|     3|
|     4|
+------+
only showing top 5 rows



DataFrame[number: bigint]

In [3]:
divBy3.show()

+------+
|number|
+------+
|     0|
|     3|
|     6|
|     9|
|    12|
|    15|
|    18|
|    21|
|    24|
|    27|
|    30|
|    33|
|    36|
|    39|
|    42|
|    45|
|    48|
|    51|
|    54|
|    57|
+------+
only showing top 20 rows



In [4]:
#working with CSV file
flightdata = spark.read.option("inferSchema","true").option("header","true").csv("2015-summary.csv")
flightdata.show()
flightdata.printSchema()

#sort ascending
sorted_flightdata = flightdata.sort("count")
sorted_flightdata.show()

#sort descending
from pyspark.sql.functions import col,desc
sorted_flightdata_desc = flightdata.sort(desc("count"))
sorted_flightdata_desc.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|   15|
|       United States|            Croatia|    1|
|       United States|            Ireland|  344|
|               Egypt|      United States|   15|
|       United States|              India|   62|
|       United States|          Singapore|    1|
|       United States|            Grenada|   62|
|          Costa Rica|      United States|  588|
|             Senegal|      United States|   40|
|             Moldova|      United States|    1|
|       United States|       Sint Maarten|  325|
|       United States|   Marshall Islands|   39|
|              Guyana|      United States|   64|
|               Malta|      United States|    1|
|            Anguilla|      United States|   41|
|             Bolivia|      United States|   30|
|       United States|           Paraguay|    6|
|             Algeri

In [5]:
#filter data
#1. using sql code
flightdata.createOrReplaceTempView("flighTable")
spark.sql("""SELECT * FROM flighTable WHERE count > 10""").show()
#2. using where command
flightdata.where(col("count")>10).show()
#3. Using filter
flightdata.filter(col("count")>10).show()

#all returns the same

#other Examples
flightdata.filter(col("DEST_COUNTRY_NAME")=="United States").show()

#more filter
flightdata.filter((col("DEST_COUNTRY_NAME")=="United States") | (col("ORIGIN_COUNTRY_NAME")=="United States")).show()

flightdata.filter((col("DEST_COUNTRY_NAME")=="United States") & (col("ORIGIN_COUNTRY_NAME")=="United States")).show()

+--------------------+--------------------+-----+
|   DEST_COUNTRY_NAME| ORIGIN_COUNTRY_NAME|count|
+--------------------+--------------------+-----+
|       United States|             Romania|   15|
|       United States|             Ireland|  344|
|               Egypt|       United States|   15|
|       United States|               India|   62|
|       United States|             Grenada|   62|
|          Costa Rica|       United States|  588|
|             Senegal|       United States|   40|
|       United States|        Sint Maarten|  325|
|       United States|    Marshall Islands|   39|
|              Guyana|       United States|   64|
|            Anguilla|       United States|   41|
|             Bolivia|       United States|   30|
|Turks and Caicos ...|       United States|  230|
|               Italy|       United States|  382|
|       United States|Federated States ...|   69|
|       United States|              Russia|  161|
|            Pakistan|       United States|   12|


#### ADDING NEW Column

In [6]:
#adding new columns in Data Frame
flightdata.withColumn("newCol",col("count") + 10).show()
#OR
#flightdata.select("DEST_COUNTRY_NAME","ORIGIN_COUNTRY_NAME","count",(col("count") + 10).alias("newCol2")).show()
#or
#flightdata.select("*",(col("count") + 10).alias("newCol2")).show()

+--------------------+-------------------+-----+------+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|newCol|
+--------------------+-------------------+-----+------+
|       United States|            Romania|   15|    25|
|       United States|            Croatia|    1|    11|
|       United States|            Ireland|  344|   354|
|               Egypt|      United States|   15|    25|
|       United States|              India|   62|    72|
|       United States|          Singapore|    1|    11|
|       United States|            Grenada|   62|    72|
|          Costa Rica|      United States|  588|   598|
|             Senegal|      United States|   40|    50|
|             Moldova|      United States|    1|    11|
|       United States|       Sint Maarten|  325|   335|
|       United States|   Marshall Islands|   39|    49|
|              Guyana|      United States|   64|    74|
|               Malta|      United States|    1|    11|
|            Anguilla|      United States|   41|

In [7]:
#Droping a column
flightdata.select("DEST_COUNTRY_NAME",(col("count") + 10).alias("newCol2")).show()

+--------------------+-------+
|   DEST_COUNTRY_NAME|newCol2|
+--------------------+-------+
|       United States|     25|
|       United States|     11|
|       United States|    354|
|               Egypt|     25|
|       United States|     72|
|       United States|     11|
|       United States|     72|
|          Costa Rica|    598|
|             Senegal|     50|
|             Moldova|     11|
|       United States|    335|
|       United States|     49|
|              Guyana|     74|
|               Malta|     11|
|            Anguilla|     51|
|             Bolivia|     40|
|       United States|     16|
|             Algeria|     14|
|Turks and Caicos ...|    240|
|       United States|     11|
+--------------------+-------+
only showing top 20 rows



#### Statistics

In [8]:
from pyspark.sql.functions import mean,min,max,stddev_pop,stddev_samp,covar_pop,covar_samp,corr,count,sum,countDistinct


In [9]:
#Find the maximum number of flights from Origin country to Destination country by reading flightdata.
flightdata.select(max("count")).show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [10]:
#Count those rows where number of flights going from Origin country to Destination country are greater than 10 by reading flightdata again.
flightdata.filter(col("count")>10).select(count("count")).show()
#NOT flightdata.select(count("count")).filter(col("count")>10).show()

+------------+
|count(count)|
+------------+
|         208|
+------------+



In [11]:
#Count total number of flights having destination country name is United States.
flightdata.filter((col("DEST_COUNTRY_NAME")=="United States")).select(sum("count")).show()

+----------+
|sum(count)|
+----------+
|    411352|
+----------+



#### GroupBy

In [12]:
dest_Country_count = flightdata.groupBy("DEST_COUNTRY_NAME").agg(sum("count"))
dest_Country_count.show()

#OR
#dest_Country_count = flightdata.groupBy("DEST_COUNTRY_NAME").agg({"count":"sum"})

+--------------------+----------+
|   DEST_COUNTRY_NAME|sum(count)|
+--------------------+----------+
|            Anguilla|        41|
|              Russia|       176|
|            Paraguay|        60|
|             Senegal|        40|
|              Sweden|       118|
|            Kiribati|        26|
|              Guyana|        64|
|         Philippines|       134|
|            Djibouti|         1|
|            Malaysia|         2|
|           Singapore|         3|
|                Fiji|        24|
|              Turkey|       138|
|                Iraq|         1|
|             Germany|      1468|
|              Jordan|        44|
|               Palau|        30|
|Turks and Caicos ...|       230|
|              France|       935|
|              Greece|        30|
+--------------------+----------+
only showing top 20 rows



#### Save Data in CSV

In [13]:
dest_Country_count.write.format("csv").option("header","true").save("dest_Country")

AnalysisException: path file:/F:/Koblenz/Study/2nd semester/Big Data/Exam 2/DataFrames/dest_Country already exists.

In [ ]:
#Using coal
dest_Country_count_merged = dest_Country_count.coalesce(1)
dest_Country_count_merged.write.format("csv").option("header","true").save("dest_Country_coal")

### Another Example

In [67]:
#retaildata = spark.read.option("inferSchema","true").option("header","true").csv("online-retail-dataset.csv")

#For date and time while loading the csv use timestamp

retaildata = spark.read.option("inferSchema","true").option("header","true").option("timestampFormat", "dd-MM-yyyy HH:mm:ss").csv("online-retail-dataset.csv")
retaildata.show()
retaildata.printSchema()



+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [15]:
#Which item was bought most (total)
most_bought = retaildata.groupBy(col("StockCode")).agg(sum(col("Quantity")).alias("sum_quantity")).sort(desc("sum_quantity"))
most_bought.show(5)
retaildata.select("StockCode","Description").where(col("StockCode") == "22197").dropDuplicates().show()



+---------+------------+
|StockCode|sum_quantity|
+---------+------------+
|    22197|       56450|
|    84077|       53847|
|   85099B|       47363|
|   85123A|       38830|
|    84879|       36221|
+---------+------------+
only showing top 5 rows

+---------+--------------------+
|StockCode|         Description|
+---------+--------------------+
|    22197|SMALL POPCORN HOLDER|
|    22197|      POPCORN HOLDER|
+---------+--------------------+



In [16]:
#Which one was bought most in the USA?
most_bought = retaildata.where(col("Country") == "USA").groupBy(col("StockCode")).agg(sum(col("Quantity")).alias("sum_quantity")).sort(desc("sum_quantity")).show(5)
retaildata.select("StockCode","Description").where(col("StockCode") == "23366").dropDuplicates().show()

+---------+------------+
|StockCode|sum_quantity|
+---------+------------+
|    23366|          88|
|    20975|          72|
|    21121|          48|
|    21124|          48|
|    21123|          48|
+---------+------------+
only showing top 5 rows

+---------+--------------------+
|StockCode|         Description|
+---------+--------------------+
|    23366|SET 12 COLOUR PEN...|
|    23366|SET 12 COLOURING ...|
|    23366|SET 12 COLOURING ...|
+---------+--------------------+



In [17]:
#Which was the lowest invoice (>0), which one the highest?
retaildata.groupBy(col("InvoiceNo")).agg(sum(col("Quantity") * col("UnitPrice")).alias("Price")).where(col("Price") > 0).sort("Price").show(5)
retaildata.groupBy(col("InvoiceNo")).agg(sum(col("Quantity") * col("UnitPrice")).alias("Price")).sort(desc("Price")).show(5)

+---------+-----+
|InvoiceNo|Price|
+---------+-----+
|   570554| 0.38|
|   567869|  0.4|
|   539441| 0.42|
|   573589| 0.55|
|   542736| 0.55|
+---------+-----+
only showing top 5 rows

+---------+------------------+
|InvoiceNo|             Price|
+---------+------------------+
|   581483|          168469.6|
|   541431|           77183.6|
|   574941| 52940.93999999999|
|   576365|50653.909999999996|
|   556444|           38970.0|
+---------+------------------+
only showing top 5 rows



In [18]:
#Add a column which displays whether an item was purchased in Germany.
retaildata.withColumn("IF Germany",col("Country") == "Germany").show(5)

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|IF Germany|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|     false|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     false|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|     false|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     false|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|     false|
+---------+---------+--------------------+--------+--------------+---------+----------+-

In [20]:
#GroupBy using 2 columns
retail_selection = retaildata.groupBy("StockCode", "Country").agg(sum("Quantity").alias("Quantity_total"))
retail_selection.show()

+---------+--------------+--------------+
|StockCode|       Country|Quantity_total|
+---------+--------------+--------------+
|    22154|United Kingdom|          3646|
|    22478|United Kingdom|          1423|
|    22844|United Kingdom|           916|
|    21788|United Kingdom|           867|
|    22042|United Kingdom|           190|
|    22539|        France|           312|
|    85078|         Italy|            24|
|   85034C|         Italy|             4|
|    22389|United Kingdom|           202|
|    85087|United Kingdom|           304|
|   35809A|United Kingdom|           291|
|    84821|United Kingdom|           821|
|    22837|         Spain|            16|
|    21030|        Cyprus|            10|
|    84992|        Sweden|           240|
|    22713|   Switzerland|            12|
|    21989|       Belgium|            36|
|    21080|       Germany|           385|
|   84661C|United Kingdom|            71|
|   82613D|        Cyprus|            20|
+---------+--------------+--------

In [21]:
#Add a column which shows the total amount of the corresponding invoice.
total_price = retaildata.groupBy(col("InvoiceNo")).agg(sum(col("Quantity") * col("UnitPrice")).alias("Price"))
retaildata.join(total_price,["InvoiceNo"]).show(5)


+---------+---------+--------------------+--------+---------------+---------+----------+--------------+-----+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|Price|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+-----+
|   536596|    21624|VINTAGE UNION JAC...|       1|12/1/2010 17:29|     5.95|      null|United Kingdom|38.09|
|   536596|    22900| SET 2 TEA TOWELS...|       1|12/1/2010 17:29|     2.95|      null|United Kingdom|38.09|
|   536596|    22114|HOT WATER BOTTLE ...|       1|12/1/2010 17:29|     3.95|      null|United Kingdom|38.09|
|   536596|    21967|PACK OF 12 SKULL ...|       1|12/1/2010 17:29|     0.29|      null|United Kingdom|38.09|
|   536596|   84926A|WAKE UP COCKEREL ...|       4|12/1/2010 17:29|     1.25|      null|United Kingdom|38.09|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+-----+
only showi

In [22]:
#How many German customers spent more than $10?
per_Customer_cost = retaildata.where(col("Country") == "Germany").groupBy(col("InvoiceNo")).agg(sum(col("Quantity") * col("UnitPrice")).alias("Paid"))
per_Customer_cost.where(col("Paid") > 10).count()

455

In [23]:
#Sort the German customers with respect to their total invoice in descending order.
per_Customer_cost.sort(desc("Paid")).show()

+---------+------------------+
|InvoiceNo|              Paid|
+---------+------------------+
|   552978| 9341.260000000004|
|   564856|4257.0599999999995|
|   543737|2355.9500000000003|
|   581179|2238.5099999999993|
|   550137|           2221.02|
|   540458|2169.3899999999994|
|   569640| 2145.380000000001|
|   576910|2037.6899999999996|
|   538174|1982.4099999999999|
|   557466|1965.7400000000002|
|   571065|           1881.86|
|   578472|           1750.46|
|   541965|           1716.62|
|   571328|           1697.98|
|   537201|1631.3000000000009|
|   539395|1484.2499999999998|
|   547822|            1444.8|
|   550527|1411.6699999999998|
|   569844|1377.5499999999993|
|   571824|1376.4299999999998|
+---------+------------------+
only showing top 20 rows



In [24]:
#changing column name after groupby
df = retaildata.where(col("Country") == "United Kingdom").groupBy(col("CustomerID")).agg(countDistinct("InvoiceNo"))
df.show()
df.withColumnRenamed("CustomerID","Desh")

+----------+----------------+
|CustomerID|count(InvoiceNo)|
+----------+----------------+
|     15619|               1|
|     17389|              43|
|     12940|               4|
|     13623|               7|
|     14450|               3|
|     15727|               7|
|     15790|               1|
|     13285|               4|
|     14570|               2|
|     16574|               1|
|     13832|               2|
|     16503|               5|
|     15957|               1|
|     17679|               2|
|     17420|               3|
|     15447|               1|
|     16386|               2|
|     18024|               3|
|     16861|               3|
|     16339|               1|
+----------+----------------+
only showing top 20 rows



DataFrame[Desh: int, count(InvoiceNo): bigint]

In [71]:
#THIS CODE IS NOT WORKING BECAUSE IT IS UNABLE TO READ DATETIME FROM CSV
#How many orders did customers perform at which hour?
#here the CSV is not reading the timestamp therefore we read it again by manually giving datatypes

retaildata_date = spark.read.csv(
    path = "online-retail-dataset.csv",
    sep = ",",
    header=True,
    quote='"',
    schema="InvoiceNo STRING ,StockCode STRING ,Description STRING ,Quantity INT ,InvoiceDate TIMESTAMP ,UnitPrice DOUBLE ,CustomerID INT ,Country STRING")

retaildata_date.printSchema()
retaildata_date.show()

#from pyspark.sql.functions import expr
#retaildata.selectExpr("date(InvoiceDate)").show()
#retail_hour = retaildata.selectExpr("hour(InvoiceDate) as InvoiceHour", "InvoiceNo").distinct().groupBy("InvoiceHour").agg(expr("count(InvoiceNo) as NoInovoices")).orderBy("InvoiceHour")
#retail_hour.show()

retaildata_date.createOrReplaceTempView("retaildata_date")
spark.sql("""SELECT hour(InvoiceDate) as InvoiceHour, count(DISTINCT InvoiceNo) as NoInovoices
            FROM retaildata_date 
            GROUP BY InvoiceHour
            ORDER BY InvoiceHour""").show()


root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: integer (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-----------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|       null|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|       null|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|       null|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|       n

In [79]:
#How frequently was each product bought in the different countries?

retaildata.groupBy(col("StockCode"),col("Country")).agg(sum("Quantity")).orderBy("StockCode").show()

retaildata_date.createOrReplaceTempView("retaildata")
spark.sql("""SELECT StockCode, Country, sum(Quantity) as Quantity
            FROM retaildata 
            GROUP BY StockCode, Country
            GROUPING SETS ((Country,StockCode),(Country),(StockCode), ())   
            ORDER BY StockCode""").show()
#here is a datacube while grouping set

+---------+--------------+-------------+
|StockCode|       Country|sum(Quantity)|
+---------+--------------+-------------+
|    10002|United Kingdom|          615|
|    10002|          EIRE|           12|
|    10002|        France|          372|
|    10002|         Japan|            1|
|    10002|   Switzerland|           12|
|    10002|       Germany|            1|
|    10002|         Spain|           24|
|    10080|United Kingdom|          495|
|    10120|        France|           10|
|    10120|United Kingdom|          183|
|   10123C|United Kingdom|          -13|
|   10123G|United Kingdom|          -38|
|   10124A|United Kingdom|           16|
|   10124G|United Kingdom|           17|
|    10125|        France|          470|
|    10125|       Germany|          120|
|    10125|   Switzerland|           20|
|    10125|United Kingdom|          686|
|    10133|        Israel|           20|
|    10133|          EIRE|           40|
+---------+--------------+-------------+
only showing top

In [110]:
#not Working

from pyspark.sql.functions import to_timestamp,quarter,year
dataframeNoNull = retaildata.na.drop().withColumn("Date", to_date(col("InvoiceDate"),"MM-dd-yyyy"))
dataframeNoNull.show()
dataframeNoNull.printSchema()
dataframeNoNull.withColumn("Quarter", quarter(col("Date"))).withColumn("Year", year(col("Date"))).show()


def generalizedCountry(country):
    switcher = {
        "Austria": "EU",
        "Germany": "EU",
        "United Kingdom": "UK",
    }
    return switcher.get(country, "Invalid")        

dataframeNoNull.withColumn("Region", generalizedCountry(col("Country"))).show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|Date|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+----+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|null|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|null|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|null|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|null|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|null|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|null|
|   536365|    21730|GLASS S

TypeError: unhashable type: 'Column'

### Assignment Question 

In [25]:
dataframe = spark.read.option("inferSchema","true").option("header","true").csv("DEvideos.csv")
dataframe.show(5)

+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+
|   video_id|trending_date|               title|       channel_title|category_id|       publish_time|                tags|  views| likes|dislikes|
+-----------+-------------+--------------------+--------------------+-----------+-------------------+--------------------+-------+------+--------+
|LgVi6y5QIjM|     17.14.11|Sing zu Ende! | G...|           inscope21|         24|2017-11-13 18:08:49|"inscope21|""sing...| 252786| 35885|     230|
|Bayt7uQith4|     17.14.11|Kinder ferngesteu...|LUKE! Die Woche u...|         23|2017-11-12 23:30:01|"Kinder|""fernges...| 797196| 53576|     302|
|1ZAPwfrtAFY|     17.14.11|The Trump Preside...|     LastWeekTonight|         24|2017-11-13 08:30:00|"last week tonigh...|2418783| 97190|    6146|
|AHtypnRk7JE|     17.14.11| Das Fermi-Paradoxon|   100SekundenPhysik|         27|2017-11-12 16:00:01|"Physik|""Wissens

In [26]:
videoStats = dataframe.select("video_id", "trending_date", "title", "views")
videoStats.show()

+-----------+-------------+--------------------+-------+
|   video_id|trending_date|               title|  views|
+-----------+-------------+--------------------+-------+
|LgVi6y5QIjM|     17.14.11|Sing zu Ende! | G...| 252786|
|Bayt7uQith4|     17.14.11|Kinder ferngesteu...| 797196|
|1ZAPwfrtAFY|     17.14.11|The Trump Preside...|2418783|
|AHtypnRk7JE|     17.14.11| Das Fermi-Paradoxon| 380247|
|ZJ9We4bjcg0|     17.14.11|18 SONGS mit Kell...| 822213|
|xapGFgWqtg4|     17.14.11|Geld verdienen mi...|  32709|
|EIM7RMe39JY|     17.14.11|Kanaken auf der K...| 308683|
|PaWTaj6Iie0|     17.14.11|3 unbekannte Gesi...| 181660|
|GHct2dGNLks|     17.14.11|Antoine leiht Aut...| 369173|
|aZYSFByDGkg|     17.14.11|Legenden: So wird...|  62418|
|2hu_evXPpMM|     17.14.11|Dagi Bee wird Hei...| 228574|
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...| 748374|
|2Zp-Qm3wJkA|     17.14.11|JP Performance - ...| 465883|
|mmMPprcmJeA|     17.14.11|Diese Schlafposit...| 109571|
|3U51cVIqulM|     17.14.11|ARAB

In [27]:
#Now after selecting the columns we are interested in, we would like to create a new column inside "videoStats" dataframe with the name "new" that contains the views divided by 100.

videoStats.withColumn("new",col("views")/100).show()

+-----------+-------------+--------------------+-------+--------+
|   video_id|trending_date|               title|  views|     new|
+-----------+-------------+--------------------+-------+--------+
|LgVi6y5QIjM|     17.14.11|Sing zu Ende! | G...| 252786| 2527.86|
|Bayt7uQith4|     17.14.11|Kinder ferngesteu...| 797196| 7971.96|
|1ZAPwfrtAFY|     17.14.11|The Trump Preside...|2418783|24187.83|
|AHtypnRk7JE|     17.14.11| Das Fermi-Paradoxon| 380247| 3802.47|
|ZJ9We4bjcg0|     17.14.11|18 SONGS mit Kell...| 822213| 8222.13|
|xapGFgWqtg4|     17.14.11|Geld verdienen mi...|  32709|  327.09|
|EIM7RMe39JY|     17.14.11|Kanaken auf der K...| 308683| 3086.83|
|PaWTaj6Iie0|     17.14.11|3 unbekannte Gesi...| 181660|  1816.6|
|GHct2dGNLks|     17.14.11|Antoine leiht Aut...| 369173| 3691.73|
|aZYSFByDGkg|     17.14.11|Legenden: So wird...|  62418|  624.18|
|2hu_evXPpMM|     17.14.11|Dagi Bee wird Hei...| 228574| 2285.74|
|2kyS6SvSYSE|     17.14.11|WE WANT TO TALK A...| 748374| 7483.74|
|2Zp-Qm3wJ

In [28]:
#Please write the line of code to calculate the mean of the views:
videoStats.agg(mean("views")).show()
videoStats.select(mean("views")).show()

+-----------------+
|       avg(views)|
+-----------------+
|603455.3184378061|
+-----------------+

+-----------------+
|       avg(views)|
+-----------------+
|603455.3184378061|
+-----------------+



In [29]:
#Please write the line of code to calculate the standard deviation of the population of the views:
videoStats.agg(stddev_pop("views")).show()
videoStats.select(stddev_pop("views")) .show()

+-----------------+
|stddev_pop(views)|
+-----------------+
|2348934.009915401|
+-----------------+

+-----------------+
|stddev_pop(views)|
+-----------------+
|2348934.009915401|
+-----------------+



In [30]:
#Please write the line of code to find the maximum of the views:
videoStats.select(max("views")).show()

+----------+
|max(views)|
+----------+
| 113876217|
+----------+



In [31]:
#Group the dataframe by the "trending_date" column and aggregate by the mean of the "views" column. The result should be put in a new dataframe named "videoStatGroup"

videoStatGroup = videoStats.groupBy("trending_date").agg(mean("views")).show()

+-------------+-----------------+
|trending_date|       avg(views)|
+-------------+-----------------+
|     18.08.05|       823764.165|
|     17.20.11|        698332.21|
|     17.09.12|       865658.605|
|     18.14.02|       385133.955|
|     18.20.03| 834208.824120603|
|     18.04.05|        641309.99|
|     18.21.02|727736.6683417085|
|     18.30.04|665931.1306532663|
|     18.06.05|        622344.45|
|     18.11.02|566525.9492385787|
|     18.15.03|566690.5075376885|
|     18.08.06|930508.5326633166|
|     18.09.06|       847713.695|
|     18.08.01|        529929.55|
|     18.20.01|        634982.22|
|     18.27.02| 605797.525510204|
|     18.15.01|       714033.835|
|     17.18.11|       429058.245|
|     17.24.12|         667388.9|
|     18.27.01|       466006.955|
+-------------+-----------------+
only showing top 20 rows



### Exam Questions

In [32]:
#replace all the null values from the column Run_Type with the value "Other"
DF.na.fill("other", subset = ["Run_Type"]).show()
DF.fillna("other", subset = ["Run_Type"]).show()

#drop all the rows which contain a null value in the "Town" column.
DF.dropna("all", subset = ["Town"])
DF.dropna("any", subset = ["Town"])
DF.na.drop("all", subset = ["Town"])
DF.na.drop("any", subset = ["Town"])

#replace all the null values in the column "School_Year" with the value 0.
DF.na.fill(value = 0, subset = ["School_Year"]).show()
DF.fillna(value = 0, subset = ["School_Year"]).show()

#compute how many breakdowns occurred each year in the town of Brooklyn. Results should look like this:
DF1 = DF.where(col("Town") == "Brooklyn Kingdom").groupBy(col("School year")).agg(countDistinct("InvoiceNo"))
DF1.withColumnRenamed("School year","Year")

#Enter the line of code that drops all rows containing a missing value in any column:
DF.dropna("any")

#Define the function that returns a dataframe containing the means for each workclass

def calculate_mean(dataframe):
    return dataframe.groupBy('workclass').mean()

#Define the function that returns a dataframe containing the variance for each workclass.

def calculate_variance(dataframe):
    return dataframe.groupBy('workclass').covar_pop()

NameError: name 'DF' is not defined

In [ ]:
datacube = 